In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from pathlib import Path

In [ ]:
datapath = Path('/kaggle/input/whale2-cropped-dataset')

train_df = pd.read_csv(
    datapath / 'train2.csv',
    usecols=['image', 'species', 'individual_id'],
    dtype={
        'image': 'str',
        'species': 'str',
        'individual_id': 'str',
    },
)

train_df.head()

In [ ]:
test_df = pd.read_csv(
    datapath / 'test2.csv',
    usecols=['image', 'predictions', 'box'],
    dtype={
        'image': 'str',
        'predictions': 'str',
        'box': 'str',
    },
)
test_df.drop(['box'], axis=1, inplace=True)
test_df['predictions'] = np.nan
test_df.head()

In [ ]:
train_df['species'].replace({
    'bottlenose_dolpin': 'bottlenose_dolphin',
    'kiler_whale': 'killer_whale',
    'beluga': 'beluga_whale',
    'globis': 'globis_whale',
},inplace =True)

train_df['class'] = train_df['species'].apply(lambda x: x.split('_')[-1])

train_df.head()


In [ ]:
train_df.isna().sum()

In [ ]:
labels = train_df['species']
filenames = train_df['image']

labels = np.array(labels)

label_encoder = LabelEncoder()
labels_enc = label_encoder.fit_transform(labels)

# One hot vector representation of labels
y_labels_one_hot = to_categorical(labels_enc, dtype='int64')

y_labels_one_hot.shape

In [ ]:
from glob import glob
import os
import random

def serialize_example(image, filename, label):

    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[str.encode(filename)])),
#        'label': tf.train.Feature(float_list=tf.train.FloatList(value=label))
        'label': tf.train.Feature(int64_list = tf.train.Int64List(value=label))
#        'label': tf.train.Feature(int64_list = tf.train.Int64List(value = tf.one_hot(label, depth=len(labels), on_value=1, off_value=0)))
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def make_tfrecords(path, files_list, labels, record_file='/kaggle/working/images.tfrecords'):
    count = 0
    with tf.io.TFRecordWriter(record_file) as writer:
        tmp = list(zip(files_list, labels))
        random.shuffle(tmp)
        files_list, labels = zip(*tmp)    
        for filename, label in zip(files_list, labels):
            image_string = open(''.join([path, '/', filename]), 'rb').read()
            image_decoded = tf.image.decode_jpeg(image_string)
            image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
            image_encoded = tf.image.encode_jpeg(tf.cast(image_resized, tf.uint8))
            image_bytes = image_encoded.numpy()
            tf_example = serialize_example(image_bytes, filename, label)
            writer.write(tf_example)
            if count % 5000 == 0:
                print('step : {}'.format(count))
            count = count + 1

In [ ]:
def _parse_image_function(example):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'filename': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([NUM_CLASSES], tf.int64),
    }

    features = tf.io.parse_single_example(example, image_feature_description)
    image = tf.image.decode_jpeg(features['image'], channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    
    filename = features['filename'] # ignore for training

    label = tf.cast(features['label'], tf.int64)

    return image, label

def read_dataset(filename, batch_size):
    dataset = tf.data.TFRecordDataset(filename)
    dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(500)
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [ ]:
def _parse_image_function_v1(example):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'filename': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([NUM_CLASSES], tf.int64),
    }

    features = tf.io.parse_single_example(example, image_feature_description)
    image = tf.image.decode_jpeg(features['image'], channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    
    filename = features['filename']

    label = tf.cast(features['label'], tf.int64)

    return image, filename, label

def read_dataset_v1(filename, batch_size):
    dataset = tf.data.TFRecordDataset(filename)
    dataset = dataset.map(_parse_image_function_v1, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(500)
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [ ]:
import tensorflow as tf

def build_model(num_classes, img_size=224):
    input = tf.keras.layers.Input(shape=(img_size, img_size, 3))
    model = tf.keras.applications.EfficientNetB3(include_top=False, input_tensor=input, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = tf.keras.layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(input, output, name="EfficientNet")
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])
    return model

In [ ]:
import math
from glob import glob

def use_tfrecords(path):
    dataset = read_dataset('/kaggle/working/images.tfrecords', BATCH_SIZE)

    model = build_model(NUM_CLASSES, IMG_SIZE)
    model.fit(dataset, 
            batch_size=BATCH_SIZE, 
            epochs=20, 
            steps_per_epoch=math.ceil(NUM_TRAIN_IMAGES/BATCH_SIZE), 
            verbose=1,
           )
    return model

In [ ]:
train_path = '/kaggle/input/whale2-cropped-dataset/cropped_train_images/cropped_train_images/'
test_path = '/kaggle/input/whale2-cropped-dataset/cropped_test_images/cropped_test_images/'

NUM_TRAIN_IMAGES, NUM_CLASSES = y_labels_one_hot.shape
NUM_TEST_IMAGES = test_df.shape[0]

NUM_FEATURES = 1536

NUM_NEIGHBORS = 5

IMG_SIZE = 80

BATCH_SIZE = 64

In [ ]:
!rm /kaggle/working/train_images.tfrecords
!rm /kaggle/working/test_images.tfrecords

make_tfrecords(train_path, 
               files_list = np.array(train_df['image']), 
               labels = y_labels_one_hot, 
               record_file = '/kaggle/working/train_images.tfrecords')

make_tfrecords(test_path, 
               files_list = np.array(test_df['image']), 
               labels = np.zeros([NUM_TEST_IMAGES, NUM_CLASSES], dtype='int64'), 
               record_file = '/kaggle/working/test_images.tfrecords')

In [ ]:
model = use_tfrecords(path)

model.save('/kaggle/working/myModel_cropped.h5')

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/whale2-data/myModel_cropped.h5')

In [ ]:
layer_name = 'avg_pool'
intermediate_layer_model = tf.keras.Model(inputs=model.input, 
                                       outputs=model.get_layer(layer_name).output
                                      )

In [ ]:
dataset = read_dataset_v1('/kaggle/input/whale2-data/train_images_cropped.tfrecords', BATCH_SIZE)

j = 0
embeddings_dict = {}
for image,filename,label in dataset.take(int(math.ceil((NUM_TRAIN_IMAGES / BATCH_SIZE)))):
    embeddings = intermediate_layer_model(np.array(image))
    embeddings_dict.update(zip(np.array(filename).astype(str),np.array(embeddings).tolist()))
    
    j = j + 1
    if j%100 == 0:
        print('Reads : {}'.format(j*BATCH_SIZE))
        
len(embeddings_dict)

In [ ]:
import pickle

with open('/kaggle/working/embeddings_dict.pkl', 'wb') as f:
    pickle.dump(embeddings_dict, f)

In [ ]:
import pickle

with open('/kaggle/input/whale2-data/embeddings_dict.pkl', 'rb') as f:
    embeddings_dict = pickle.load(f)

In [ ]:
from sklearn.neighbors import NearestNeighbors

embeddings_keys = list(embeddings_dict.keys())
embeddings_vals = list(embeddings_dict.values())

knn = NearestNeighbors(n_neighbors=NUM_NEIGHBORS, metric="cosine")
knn.fit(embeddings_vals)

In [ ]:
dataset = read_dataset_v1('/kaggle/input/whale2-data/test_images_cropped.tfrecords', BATCH_SIZE)

#keys = {tuple(np.array(test_df['image']))}
#indices_dict = dict.fromkeys(keys)
indices_dict = {}
j = 0
for image,filename,label in dataset.take(int(math.ceil((NUM_TEST_IMAGES / BATCH_SIZE)))):
    embeddings = intermediate_layer_model(np.array(image))
    _, indices = knn.kneighbors(embeddings)
    indices_list = indices.tolist()
    indices_dict.update(zip(np.array(filename).astype(str),indices_list))
    
    j = j + BATCH_SIZE
    if j%100 == 0:
        print('Reads : {}'.format(j*BATCH_SIZE))
    
print(len(indices_dict))

In [ ]:
import pickle

with open('/kaggle/working/indices_dict.pkl', 'wb') as f:
    pickle.dump(indices_dict, f)

In [ ]:
import pickle

with open('/kaggle/input/whale2-data/indices_dict.pkl', 'rb') as f:
    indices_dict = pickle.load(f)

In [ ]:
#embeddings_list = np.array(list(embeddings_dict.keys()))
indices_dict = {}
for k in indices_dict.keys():
    _, indices = knn.kneighbors(np.array(embeddings_dict['00021adfb725ed.jpg']).reshape(1,-1))
    indices_list = indices.tolist()

#you embeddings_list[tuple(indices_list)]

In [ ]:
matches_dict = {}
for k in indices_dict.keys():
    matches = train_df[train_df['image'].isin(filenames[indices_dict[k]].flatten())]['individual_id']
    matches_dict.update({k:(' '.join(matches))})
len(matches_dict)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img=mpimg.imread('/kaggle/input/happy-whale-and-dolphin/test_images/0006287ec424cb.jpg')

img_resized = tf.image.resize(img, [IMG_SIZE,IMG_SIZE])

img_np = np.array(img_resized).reshape(
      (1, img_resized.shape[0], img_resized.shape[1], 3)).astype(np.uint8)

embedding = np.array(intermediate_layer_model(img_np))

_, indices = knn.kneighbors(embedding.reshape(1,-1))
indices_list = indices.tolist()
indices_list

In [ ]:
embeddings_keys = np.array(list(embeddings_dict.keys()))
embeddings_vals = np.array(list(embeddings_dict.values()))

indices_keys = np.array(list(indices_dict.keys()))
indices_vals = np.array(list(indices_dict.values()))

In [ ]:
matches_dict = {}
for k in indices_dict.keys():
    matches = embeddings_keys[list(indices_dict[k])]
    matches_ids = train_df[train_df['image'].isin(matches.flatten())]['individual_id']
    matches_dict.update({k:(' '.join(matches_ids))})
len(matches_dict)


In [ ]:
for k in matches_dict.keys():
    test_df.loc[test_df['image'] == k,'predictions'] = matches_dict[k]

In [ ]:
test_df.head()

In [ ]:
test_df.to_csv('/kaggle/working/submission.csv', sep=',', encoding='utf-8', header=True, index=False)

In [ ]:
# HOW TO predict
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img=mpimg.imread('/kaggle/input/happy-whale-and-dolphin/test_images/0006287ec424cb.jpg')

img_resized = tf.image.resize(img, [IMG_SIZE,IMG_SIZE])

img_np = np.array(img_resized).reshape(
      (1, img_resized.shape[0], img_resized.shape[1], 3)).astype(np.uint8)

y_prob = model.predict(img_np)

y_pred = np.argmax(y_prob, axis=None, out=None)

#y_pred = (y_prob == y_prob.max(axis=1)).astype(int)

top_k_values, top_k_indices = tf.nn.top_k(y_prob, k=5)

label_encoder.inverse_transform(np.array(top_k_indices).squeeze())

#top_k_indices

In [ ]:
# HOW TO save
np.save('/kaggle/working/intermediate_output.npy', intermediate_output)
intermediate_output = np.load('/kaggle/input/whale2-data/intermediate_output.npy')

In [ ]:
# HOWTO stack
intermediate_layer_model(np.stack([img_resized,img_resized], axis=0))

In [ ]:
tf.compat.v1.enable_eager_execution()

def read_dataset_new(filename, target='delay'):
    ds = tf.data.TFRecordDataset(filename)
    ds = ds.map(lambda buf: parse(buf, target=target))
    ds = ds.batch(1)
    return ds

# This should return your key values for each example.
#for features, labels in read_dataset_new(self.tf_rcrds_fl_nm):
#    features_keys = features.keys()

# This should return your tensor values if they supposed to be numeric.
for features, labels in read_dataset_new('/kaggle/working/images.tfrecords'):
    features_array = numpy.array(features)

In [ ]:
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu',input_shape=(80,80,3)))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation = "relu")) #Fully connected layer
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(60, activation = "relu")) #Fully connected layer
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(30, activation = "softmax")) #Classification layer or output layer

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model = Sequential()

    model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu',input_shape=(80,80,3)))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(256, activation = "relu")) #Fully connected layer
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(60, activation = "relu")) #Fully connected layer
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(30, activation = "softmax")) #Classification layer or output layer

    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

    model.summary()

In [ ]:
model.fit(my_training_batch_generator,
                   steps_per_epoch = int(X_train_filenames.shape[0] // batch_size),
                   epochs = 10,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(X_val_filenames.shape[0] // batch_size))

In [ ]:
print(X_val_filenames.shape[0])

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

In [ ]:
!rm -r ./runs
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source /kaggle/input/happy-whale-and-dolphin/train_images/001001f099519f.jpg
display.Image(filename='./runs/detect/exp/001001f099519f.jpg', width=600)

In [ ]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, etc.

model.conf = 0.25
model.iou = 0.45
model.classes = None

In [ ]:
from os import listdir
from os.path import isfile, join

mypath = '/kaggle/input/happy-whale-and-dolphin/train_images/'

input_images = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
print([model(file, size=640) for file in input_images[:10]])

In [ ]:
input_images = ['/kaggle/input/happy-whale-and-dolphin/train_images/00087baf5cef7a.jpg']

#print([file for file in input_images[:10]])

#results = model(input_images[:10], size=640)
results = model(imgCropped, size=640)
labels, cord_thres = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()

labels, cord_thres

In [ ]:
x = cord_thres[0][0]
y = cord_thres[0][1]
w = cord_thres[0][2]
h = cord_thres[0][3]
_ = cord_thres[0][4] # label confidence

In [ ]:
#!/usr/bin/env python3

import sys
import pyvips
import numpy as np

# Process all input files so we only incur Python startup overhead once
for filename in sys.argv[1:]:
   print(f'Processing: {filename}')

   img = pyvips.Image.new_from_file(filename, access='sequential')
   roi = img.crop(0, 150, 270, 900)
   mem_img = roi.write_to_memory()

   # Make a numpy array from that buffer object
   nparr = np.ndarray(buffer=mem_img, dtype=np.uint8,
                   shape=[roi.height, roi.width, roi.bands])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img=mpimg.imread('/kaggle/input/happy-whale-and-dolphin/test_images/dae6e56cf64588.jpg')

H,W,_ = img.shape

x,y,w,h = int(cord_thres[0][1]*W), int(cord_thres[0][0]*H), int(cord_thres[0][2]*H), int(cord_thres[0][3]*W)

imgCrop = img[x:x+w, y:y+h]

#plt.imshow(imgCrop)

mpimg.imsave('/kaggle/working/tmp.jpg', imgCrop)

%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(imgCrop)



In [ ]:
import cv2
import matplotlib.pyplot as plt

label = cord_thres[0]
img = cv2.imread('/kaggle/input/happy-whale-and-dolphin/train_images/00087baf5cef7a.jpg')

H, W, _ = img.shape
object = [int(label[0]*W/2), int(label[1]*H/2), int(label[2]*W), int(label[3]*H)]

x,y,w,h = object
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(img[y:y+h, x:x+w])
plt.show()
plt.show()

In [ ]:
results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

In [ ]:
x,y,w,h = int(cord_thres[0][0]*W), int(cord_thres[0][1]*H), int(cord_thres[0][2]*W), int(cord_thres[0][3]*H)
print('({},{},{},{})'.format(x,y,w,h))


In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('/kaggle/input/happy-whale-and-dolphin/train_images/000562241d384d.jpg')

print(img.shape)

plt.imshow(img)

plt.show()
plt.show()

In [ ]:
import tensorflow_hub as hub

In [ ]:
ALL_MODELS = {
'CenterNet HourGlass104 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1',
'CenterNet HourGlass104 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1',
'CenterNet HourGlass104 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024/1',
'CenterNet HourGlass104 Keypoints 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1',
'CenterNet Resnet50 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1',
'CenterNet Resnet50 V1 FPN Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512_kpts/1',
'CenterNet Resnet101 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1',
'CenterNet Resnet50 V2 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1',
'CenterNet Resnet50 V2 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512_kpts/1',
'EfficientDet D0 512x512' : 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
'EfficientDet D1 640x640' : 'https://tfhub.dev/tensorflow/efficientdet/d1/1',
'EfficientDet D2 768x768' : 'https://tfhub.dev/tensorflow/efficientdet/d2/1',
'EfficientDet D3 896x896' : 'https://tfhub.dev/tensorflow/efficientdet/d3/1',
'EfficientDet D4 1024x1024' : 'https://tfhub.dev/tensorflow/efficientdet/d4/1',
'EfficientDet D5 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d5/1',
'EfficientDet D6 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d6/1',
'EfficientDet D7 1536x1536' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
'SSD MobileNet v2 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2',
'SSD MobileNet V1 FPN 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v1/fpn_640x640/1',
'SSD MobileNet V2 FPNLite 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1',
'SSD MobileNet V2 FPNLite 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1',
'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1',
'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_1024x1024/1',
'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1',
'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_1024x1024/1',
'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_640x640/1',
'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_1024x1024/1',
'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
'Faster R-CNN ResNet152 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1',
'Faster R-CNN ResNet152 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1',
'Faster R-CNN ResNet152 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_800x1333/1',
'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}

In [ ]:
#@title Model Selection { display-mode: "form", run: "auto" }
model_display_name = 'CenterNet HourGlass104 Keypoints 512x512' # @param ['CenterNet HourGlass104 512x512','CenterNet HourGlass104 Keypoints 512x512','CenterNet HourGlass104 1024x1024','CenterNet HourGlass104 Keypoints 1024x1024','CenterNet Resnet50 V1 FPN 512x512','CenterNet Resnet50 V1 FPN Keypoints 512x512','CenterNet Resnet101 V1 FPN 512x512','CenterNet Resnet50 V2 512x512','CenterNet Resnet50 V2 Keypoints 512x512','EfficientDet D0 512x512','EfficientDet D1 640x640','EfficientDet D2 768x768','EfficientDet D3 896x896','EfficientDet D4 1024x1024','EfficientDet D5 1280x1280','EfficientDet D6 1280x1280','EfficientDet D7 1536x1536','SSD MobileNet v2 320x320','SSD MobileNet V1 FPN 640x640','SSD MobileNet V2 FPNLite 320x320','SSD MobileNet V2 FPNLite 640x640','SSD ResNet50 V1 FPN 640x640 (RetinaNet50)','SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)','SSD ResNet101 V1 FPN 640x640 (RetinaNet101)','SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)','SSD ResNet152 V1 FPN 640x640 (RetinaNet152)','SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)','Faster R-CNN ResNet50 V1 640x640','Faster R-CNN ResNet50 V1 1024x1024','Faster R-CNN ResNet50 V1 800x1333','Faster R-CNN ResNet101 V1 640x640','Faster R-CNN ResNet101 V1 1024x1024','Faster R-CNN ResNet101 V1 800x1333','Faster R-CNN ResNet152 V1 640x640','Faster R-CNN ResNet152 V1 1024x1024','Faster R-CNN ResNet152 V1 800x1333','Faster R-CNN Inception ResNet V2 640x640','Faster R-CNN Inception ResNet V2 1024x1024','Mask R-CNN Inception ResNet V2 1024x1024']
model_handle = ALL_MODELS[model_display_name]

print('Selected model:'+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

In [ ]:
print('loading model...')
hub_model = hub.load(model_handle)
print('model loaded!')

In [ ]:
import numpy as np

import cv2
import matplotlib.pyplot as plt

img = cv2.imread('/kaggle/input/happy-whale-and-dolphin/train_images/001001f099519f.jpg')

img_np = np.array(img).reshape(
      (1, img.shape[0], img.shape[1], 3)).astype(np.uint8)

In [ ]:
# running inference
results = hub_model(img_np)

# different object detection models have additional results
# all of them are explained in the documentation
result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

In [ ]:
result['detection_scores']

In [ ]:
import tensorflow as tf

img = load_img('/kaggle/input/happy-whale-and-dolphin/train_images/00087baf5cef7a.jpg')

print("Found %d objects." % len(result["detection_scores"]))

image_with_boxes = draw_boxes(
    img.numpy(), result["detection_boxes"],
    None, result["detection_scores"])

display_image(image_with_boxes)

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = None #list(ImageColor.colormap.values())

  try:
    font = None # ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf", 25)
  except IOError:
    print("Font not found, using default font.")
    font = None # ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [ ]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1),
 (0, 2),
 (1, 3),
 (2, 4),
 (0, 5),
 (0, 6),
 (5, 7),
 (7, 9),
 (6, 8),
 (8, 10),
 (5, 6),
 (5, 11),
 (6, 12),
 (11, 12),
 (11, 13),
 (13, 15),
 (12, 14),
 (14, 16)]



label_id_offset = 0
image_np_with_detections = img_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in result:
  keypoints = result['detection_keypoints'][0]
  keypoint_scores = result['detection_keypoint_scores'][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_detections[0])
plt.show()

In [ ]:
result